In [3]:
from docx import Document
import os
import glob
import pandas as pd

In [4]:
# Função para extrair texto de um arquivo .docx
def extract_text(docx_file):
    document = Document(docx_file)
    full_text = []
    for para in document.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [5]:
# Listas para armazenar os textos
textos_pre = []
textos_em_correcao = []
textos_pos = []

# Diretório onde os arquivos estão armazenados
diretorio = 'peticoes'

# Navegar até o diretório
os.chdir(diretorio)



In [6]:
# Verificar se o número de arquivos em cada lista é igual
arquivos_pre = sorted(glob.glob('pre-correcao*.docx'))
arquivos_em_correcao = sorted(glob.glob('durante-correcao*.docx'))
arquivos_pos = sorted(glob.glob('pos-correcao*.docx'))
assert len(arquivos_pre) == len(arquivos_em_correcao) == len(arquivos_pos), "Número de arquivos inconsistente." # verificar inconsistencias 

In [7]:
# Iterar pelos arquivos e extrair os textos
for arquivo_pre, arquivo_em_correcao, arquivo_pos in zip(arquivos_pre, arquivos_em_correcao, arquivos_pos):
    texto_pre = extract_text(arquivo_pre)
    texto_em_correcao = extract_text(arquivo_em_correcao)
    texto_pos = extract_text(arquivo_pos)
    
    textos_pre.append(texto_pre)
    textos_em_correcao.append(texto_em_correcao)
    textos_pos.append(texto_pos)

In [16]:
# Criar um DataFrame para organizar os dados
df_peticoes = pd.DataFrame({
    'texto_pre': textos_pre,
    'texto_em_correcao': textos_em_correcao,
    'texto_pos': textos_pos
})
# Exibir o DataFrame
print(df_peticoes.head())

In [ ]:
print(df_peticoes['texto_pre'])

NameError: name 'df' is not defined